In [6]:
!pip install datasets bert_score transformers

Defaulting to user installation because normal site-packages is not writeable


In [7]:
from bert_score import score as bertscore_score
import random, re

DOC_COL = "document"
REF_COL = "summary"
SPLIT_TOKEN = "|||||"
RANDOM_SEED = 42

rng = random.Random(RANDOM_SEED)

def split_docs(multidoc_str: str):
    if not isinstance(multidoc_str, str):
        return []
    return [d.strip() for d in multidoc_str.split(SPLIT_TOKEN) if d and d.strip()]

def first_sentence(text: str):
    if not isinstance(text, str):
        return ""
    text = text.strip()
    if not text:
        return ""
    idx = text.find(".")
    if idx == -1:
        return text
    text = text[:idx+1].strip()
    parts = text.split("\n")  # at most 3 pieces
    # Keep everything after the second double newline if it exists
    return parts[-1].strip()

def get_ref(data):
    res = []
    for count, doc in enumerate(data):
        if len(doc) == 0:
            print("empty seq", count)
        docs = split_docs(doc)
        chosen_doc = rng.choice(docs)
        res.append(first_sentence(chosen_doc))
    return res

def predict_bias_scores(text: str):
    if not text or not text.strip():
        return np.zeros(7, dtype=float)
    rng = random.Random(hash(text) % (2**32))
    return np.array([rng.random() * 0.2 for _ in range(7)], dtype=float)

def compute_bertscore(cands, refs, model_type="roberta-large", lang="en"):
    P, R, F1 = bertscore_score(cands, refs, lang=lang, model_type=model_type)
    return np.array([f.item() for f in F1])

In [8]:
from datasets import load_dataset
import pandas as pd

dataset_train = load_dataset("Awesome075/multi_news_parquet", split="train").to_pandas()
dataset_test = load_dataset("Awesome075/multi_news_parquet", split="test").to_pandas()
dataset_val = load_dataset("Awesome075/multi_news_parquet", split="validation").to_pandas()
dataset_test.head(5)

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...


In [9]:
import numpy as np

dataset_test[DOC_COL].replace('', np.nan, inplace=True)
dataset_test[REF_COL].replace('', np.nan, inplace=True)
dataset_test.dropna(inplace=True)
refs = get_ref(dataset_test[DOC_COL])
print(refs[2])

The Siskiyou County Board of Supervisors voted 4-1 on Tuesday to support the county's split from the state of California.


In [10]:
actual = dataset_test[REF_COL].tolist()

scores = compute_bertscore(refs, actual)
print(scores)

2025-11-05 21:36:26.840680: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 21:36:26.999964: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 21:36:32.303598: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Some weights of RobertaModel were not initialized from the model checkpoint at r

[0.86647177 0.8404299  0.87250227 ... 0.8412149  0.81790543 0.85291541]


In [11]:
print(scores.mean())

0.8299465036396454


In [12]:
from transformers import pipeline

# Load the model
classifier = pipeline(
    "text-classification",
    model="cirimus/modernbert-large-bias-type-classifier",
    top_k=None
)

Device set to use cuda:0
/home/tmittal/.local/lib/python3.11/site-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()


[[{'label': 'political', 'score': 0.9999349117279053}, {'label': 'educational', 'score': 0.006539540830999613}, {'label': 'socioeconomic', 'score': 0.0035005114041268826}, {'label': 'age', 'score': 0.0015208054101094604}, {'label': 'religious', 'score': 0.000706337857991457}, {'label': 'nationality', 'score': 0.0006610832060687244}, {'label': 'racial', 'score': 0.00043269211892038584}, {'label': 'gender', 'score': 0.0002153290988644585}, {'label': 'disability', 'score': 0.00019418167357798666}, {'label': 'sexuality', 'score': 0.00011688211816363037}, {'label': 'physical', 'score': 4.504854587139562e-05}], [{'label': 'sexuality', 'score': 0.9419041872024536}, {'label': 'gender', 'score': 0.13597071170806885}, {'label': 'age', 'score': 0.10291977226734161}, {'label': 'political', 'score': 0.04822717607021332}, {'label': 'educational', 'score': 0.003797091543674469}, {'label': 'racial', 'score': 0.002407519845291972}, {'label': 'disability', 'score': 0.0013916435418650508}, {'label': 'phy

In [13]:
def get_bias_scores(preds):
    return classifier(preds)

bias_scores = get_bias_scores(refs)
print(bias_scores[1])

[{'label': 'sexuality', 'score': 0.9419041872024536}, {'label': 'gender', 'score': 0.13597071170806885}, {'label': 'age', 'score': 0.10291977226734161}, {'label': 'political', 'score': 0.04822717607021332}, {'label': 'educational', 'score': 0.003797091543674469}, {'label': 'racial', 'score': 0.002407519845291972}, {'label': 'disability', 'score': 0.0013916435418650508}, {'label': 'physical', 'score': 0.0013883321080356836}, {'label': 'nationality', 'score': 0.0011251871474087238}, {'label': 'religious', 'score': 0.0007885863888077438}, {'label': 'socioeconomic', 'score': 0.0006402177968993783}]


In [14]:
def compute_neutrality(bias_output):
    scores = sorted([float(item["score"]) for item in bias_output], reverse=True)
    top3 = scores[:3] if len(scores) >= 3 else scores  # handle fewer categories
    avg_top3 = np.mean(top3)
    neutrality = (1.0 - avg_top3) ** 2
    return neutrality


neutrality_scores = [compute_neutrality(bias_score) for bias_score in bias_scores]

In [15]:
print(np.mean(neutrality_scores))

0.48599172787542383


In [20]:
for i in range(3):
    print(refs[i][:10], actual[i][:20], scores[i], neutrality_scores[i])

Voters in  – It's a race for th 0.8664717674255371 0.44002218267265963
It’s time  – It turns out Faceb 0.8404299020767212 0.3677231142683172
The Siskiy – Not a big fan of S 0.8725022673606873 0.2722241179087657
